
# Çift GPU (Multi‑GPU) Mantığı ve SyncBatchNorm Nerede Devreye Girer?

Bu notebook’ta şunları **temelden ileri seviyeye** netleştiriyoruz:

- “Çift GPU” tam olarak ne demek?
- Neden tek GPU’da SyncBatchNorm anlamsız?
- Bir model “neden birden fazla GPU ister?” (VRAM ve compute motivasyonları)
- **Data Parallel vs DDP (DistributedDataParallel)** farkı
- **SyncBatchNorm** hangi yerlerde iş görür, nerede gereksizdir?
- Pratik senaryolar: detection / segmentation / video / dev model
- Çalıştırma şablonları (`torchrun`, `LOCAL_RANK`, `DistributedSampler`)

> Not: Çoklu GPU kodları genelde notebook’tan ziyade `train.py` script’inde çalıştırılır.
> Burada “kopyala‑yapıştır” şablonlar veriyorum.



## 1) “Çift GPU” nedir?

**Çift GPU**, bilgisayarda **iki adet fiziksel ekran kartı** (2× GPU) olmasıdır.

Örnek:
- GPU0: RTX 3060
- GPU1: RTX 3060 (ya da başka bir GPU)

Bu durumda her GPU:
- Kendi **CUDA çekirdeklerine**
- Kendi **VRAM**’ine (örn. 12 GB + 12 GB)
sahiptir.

### Kritik:
- GPU’nun içindeki CUDA çekirdekleri “ikinci GPU” değildir.
- CPU da “ikinci GPU” değildir.

**Multi‑GPU** demek: *Birden fazla fiziksel GPU cihazı* demek.



## 2) Bir model neden “birden fazla GPU ister”?

İki ana sebep var:

### 2.1. VRAM (Bellek) yetmiyordur
- Görüntü çözünürlüğü yüksek (ör. 1024×1024, 4K)
- Model büyük (çok katman, çok kanal)
- Batch büyütmek istiyorsun ama VRAM yetmiyor
- 3D/video tensörleri çok büyük (B, C, T, H, W)

Bu durumda birden fazla GPU ile:
- **Veriyi paylaşarak** (data parallel) batch’i büyütürsün
- Ya da **modeli bölerek** (model parallel) tek örneği sığdırırsın

### 2.2. Compute (hesap) yetmiyordur / hız istiyorsundur
- Tek GPU ile eğitim süresi çok uzun
- Daha fazla GPU ile aynı işi daha hızlı bitirmek istersin

> Multi‑GPU, her zaman “daha hızlı” demek değildir.
> Haberleşme overhead’i (GPU‑GPU iletişimi) yüzünden bazı durumlarda verim düşebilir.



## 3) Multi‑GPU yaklaşımları: Data Parallel vs Model Parallel

### 3.1. Data Parallel (Veri paralelliği)
En yaygın yaklaşım budur.

- Modelin **aynı kopyası** her GPU’da vardır
- Dataset/batch, GPU’lar arasında bölünür
- Her GPU kendi parçasında forward/backward yapar
- Sonra gradyanlar toplanır ve tüm GPU’lar aynı ağırlıkları günceller

**Kullanım amacı:** Batch’i büyütmek ve throughput artırmak

### 3.2. Model Parallel (Model paralelliği)
- Modelin katmanları GPU’lara bölünür
- Örn. ilk yarı GPU0, ikinci yarı GPU1
- Tek bir örnek bile tek GPU’ya sığmıyorsa kullanılır

**Kullanım amacı:** Dev modeli/büyük aktivasyonları sığdırmak

> SyncBatchNorm, esas olarak **data parallel (DDP)** dünyasında önemlidir.



## 4) DataParallel (DP) vs DistributedDataParallel (DDP)

### 4.1. DataParallel (eski yol)
- Tek process, çok GPU
- Forward’da batch’i böler, sonuçları toplar
- Çoğu durumda **DDP’den daha yavaş ve daha sorunlu**

### 4.2. DDP (modern, doğru yol)
- GPU başına **1 process**
- Her process bir GPU’ya sabitlenir
- Gradient senkronizasyonu **NCCL all‑reduce** ile verimli yapılır
- Ölçeklenebilirlik daha iyi

**Özet:** Multi‑GPU eğitimde PyTorch tarafında standart = **DDP**



## 5) DDP nasıl çalışır? (Adım adım)

Varsayalım 2 GPU var.

### 5.1. Her GPU’da ayrı process
- Process0 → GPU0
- Process1 → GPU1

### 5.2. Data bölünür
- GPU0 batch’in ilk yarısını görür
- GPU1 batch’in ikinci yarısını görür

### 5.3. Her GPU kendi gradient’ini çıkarır
- Forward + backward her GPU’da ayrı

### 5.4. All‑Reduce ile gradyanlar toplanır
- GPU0 ve GPU1 gradyanlarını birbirine gönderir/ortalama alır
- Sonuç: iki GPU da **aynı** güncellenmiş ağırlıklara sahip olur

### Ana fikir:
DDP, GPU’lar arasında “ağırlık güncellemesini aynı tutmak” için sürekli senkronizasyon yapar.



## 6) SyncBatchNorm bu resimde nereye oturuyor?

### 6.1. Normal BatchNorm’un DDP’de sorunu
BatchNorm train modunda mean/var’ı **mini‑batch’ten** hesaplar.

DDP’de:
- GPU0 kendi mini‑batch’inden mean/var hesaplar
- GPU1 kendi mini‑batch’inden mean/var hesaplar

Eğer per‑GPU batch küçükse (1–4–8 gibi):
- İstatistikler gürültülü olur
- GPU’lar farklı normalize eder
- Eğitim kararsızlaşabilir (özellikle detection/segmentation)

### 6.2. SyncBatchNorm çözümü
SyncBatchNorm:
- Her GPU’nun kısmi istatistiğini çıkarır
- GPU’lar arasında **all‑reduce** ile birleştirir
- Tek bir “global” mean/var elde eder
- Her GPU aynı mean/var ile normalize eder ✅

### 6.3. Neden tek GPU’da anlamsız?
Tek GPU’da:
- Senkronize edilecek başka GPU yok
- Global = lokal
Dolayısıyla SyncBN, normal BN ile aynı davranışa iner (fayda yok).



## 7) SyncBatchNorm nerelerde kullanılır?

### 7.1. Çok yaygın kullanım alanları
- **Object Detection** (YOLO türevleri, Faster R‑CNN, RetinaNet vb. CNN‑tabanlı backbone’lar)
- **Segmentation** (U‑Net, DeepLab türevleri)
- **Video / 3D Conv** (batch genelde küçük olur)
- Büyük çözünürlükte training (VRAM yüzünden batch küçülür)

### 7.2. Ne zaman gereksiz?
- Per‑GPU batch zaten büyükse (göreli olarak)
- Tek GPU eğitim
- Transformer ağırlıklı mimariler (LayerNorm standardı)
- İletişimin aşırı pahalı olduğu altyapı (yavaş interconnect)

### 7.3. Alternatif: GroupNorm
Tek GPU + küçük batch için en pratik alternatif çoğunlukla **GroupNorm**’dur.
Batch’e bağımlı değildir, senkronizasyon gerektirmez.



## 8) “İki GPU varsa batch nasıl büyür?” (Somut örnek)

Varsayalım:
- Per‑GPU batch = 4
- GPU sayısı = 2

DDP ile efektif global batch:
- Global batch ≈ 4 × 2 = 8

Ama dikkat:
- Bu “global batch”,
- BN istatistiği açısından **SyncBN kullanmazsan** yine GPU başına 4’ten hesaplanır.

SyncBN ile:
- BN istatistiği global 8 üzerinden hesaplanır.


In [1]:
# Basit hesap: global batch = per_gpu_batch * world_size
per_gpu_batch = 4
world_size = 2
global_batch = per_gpu_batch * world_size
global_batch

8


## 9) DDP + SyncBatchNorm: doğru kurulum şablonu

**Altın kural:** SyncBN dönüşümü DDP wrap’inden önce yapılır.

Akış:
1) `dist.init_process_group(backend="nccl")`
2) `torch.cuda.set_device(local_rank)`
3) `model.cuda(local_rank)`
4) `model = nn.SyncBatchNorm.convert_sync_batchnorm(model)`
5) `model = DDP(model, device_ids=[local_rank])`
6) `DistributedSampler` ile DataLoader

Aşağıdaki script şablonunu alıp `train.py` yap:


In [2]:

ddp_syncbn_script = r'''
# train.py
import os
import torch
import torch.nn as nn
import torch.distributed as dist
from torch.nn.parallel import DistributedDataParallel as DDP
from torch.utils.data import DataLoader, DistributedSampler
from torchvision import datasets, transforms

def setup():
    dist.init_process_group(backend="nccl")
    local_rank = int(os.environ["LOCAL_RANK"])
    torch.cuda.set_device(local_rank)
    return local_rank

class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(
            nn.Conv2d(3, 32, 3, padding=1),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.Conv2d(32, 64, 3, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.AdaptiveAvgPool2d(1),
            nn.Flatten(),
            nn.Linear(64, 10),
        )
    def forward(self, x): return self.net(x)

def main():
    local_rank = setup()

    tfm = transforms.Compose([transforms.Resize((64,64)), transforms.ToTensor()])
    ds = datasets.FakeData(size=2000, image_size=(3,64,64), num_classes=10, transform=tfm)
    sampler = DistributedSampler(ds, shuffle=True)
    dl = DataLoader(ds, batch_size=8, sampler=sampler, num_workers=2, pin_memory=True)

    model = Net().cuda(local_rank)

    # SyncBN dönüşümü DDP'den ÖNCE
    model = nn.SyncBatchNorm.convert_sync_batchnorm(model)

    model = DDP(model, device_ids=[local_rank])

    opt = torch.optim.AdamW(model.parameters(), lr=1e-3)
    loss_fn = nn.CrossEntropyLoss()

    model.train()
    for epoch in range(3):
        sampler.set_epoch(epoch)
        for x, y in dl:
            x = x.cuda(local_rank, non_blocking=True)
            y = y.cuda(local_rank, non_blocking=True)
            opt.zero_grad(set_to_none=True)
            out = model(x)
            loss = loss_fn(out, y)
            loss.backward()
            opt.step()

        if local_rank == 0:
            print("epoch", epoch, "done")

    dist.destroy_process_group()

if __name__ == "__main__":
    main()
'''
print(ddp_syncbn_script[:1200] + "\n...\n" + ddp_syncbn_script[-350:])



# train.py
import os
import torch
import torch.nn as nn
import torch.distributed as dist
from torch.nn.parallel import DistributedDataParallel as DDP
from torch.utils.data import DataLoader, DistributedSampler
from torchvision import datasets, transforms

def setup():
    dist.init_process_group(backend="nccl")
    local_rank = int(os.environ["LOCAL_RANK"])
    torch.cuda.set_device(local_rank)
    return local_rank

class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(
            nn.Conv2d(3, 32, 3, padding=1),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.Conv2d(32, 64, 3, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.AdaptiveAvgPool2d(1),
            nn.Flatten(),
            nn.Linear(64, 10),
        )
    def forward(self, x): return self.net(x)

def main():
    local_rank = setup()

    tfm = transforms.Compose([transforms.Resize((64,64)), transforms.ToTensor()])
  


### Çalıştırma komutu
2 GPU için:
```bash
torchrun --nproc_per_node=2 train.py
```

4 GPU için:
```bash
torchrun --nproc_per_node=4 train.py
```

> Tek GPU’da `--nproc_per_node=1` olur ama SyncBN’nin faydası yoktur.



## 10) Sık yapılan hatalar (kısa ama kritik)

- **DataParallel kullanmak:** SyncBN/performans için önerilmez.
- `convert_sync_batchnorm`’u DDP’den sonra yapmak: yanlış.
- `DistributedSampler` kullanmamak: her GPU aynı veriyi görebilir, verim düşer.
- `model.eval()` modunda SyncBN beklemek: eval’da BN running stats kullanır, sync yok.
- Per‑GPU batch = 1 ise bile SyncBN stabilite sağlar ama iletişim maliyeti artabilir.